In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
slr = LinearRegression()

X_test = pd.read_pickle("X_test.pickle")
y_test = pd.read_pickle("y_test.pickle")


In [ ]:
# Read the synthesised data and save it as X_train and y_train
X_train = 
y_train = 

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
def model_eval(model,X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test):
    """PyML P477-480
    Plot residual plot , dots should be located around the 0 line, and have no pattern
    R_square should be close to 1
    MSE should be small
    """
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    print('R^2 train: %.3f, test: %.3f' %(r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)))
    print("R2",model.score(X_train, y_train))
    print("R2 Adj",1 - (1-model.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
    
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    print('MSE train: %.3f, test: %.3f' %(mse_train, mse_test))
    print('Over fitting' if mse_train < mse_test else 'Under fitting')
    
    plt.figure(figsize=(10,10))
    plt.scatter(y_train_pred, y_train_pred - y_train, c='steelblue', marker='o', edgecolor='white', label='Training data')
    plt.scatter(y_test_pred, y_test_pred - y_test, c='limegreen', marker='s', edgecolor='white', label='Test data')
    plt.xlabel('Predicted values')
    plt.ylabel('Residuals')
    plt.legend(loc='upper left')
    #plt.hlines(y=0, xmin=-10, xmax=350000,color='black', lw=2)
    plt.show()
    


In [ ]:
slr.fit(X_train, y_train)
model_eval(slr)